In [1]:
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)


In [52]:
assistant = client.beta.assistants.create(
    name='Math tutor',
    instructions='You are a personal math tutor. Write and run code to answer math questions.',
    tools=[{'type': 'code_interpreter'}],
    model='gpt-3.5-turbo',
)



In [53]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_QyeByGlS8DkctSed6RlUKGFG', created_at=1720044423, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [54]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content = 'Solve this problem: 3x + 11 = 14'
)

In [55]:
print(message)

Message(id='msg_Pyot3szBNKnasUrun5zSPk3d', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Solve this problem: 3x + 11 = 14'), type='text')], created_at=1720044424, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_QyeByGlS8DkctSed6RlUKGFG')


In [56]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    
    assistant_id= assistant.id,
)

In [57]:
run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
)

In [71]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id
)
print(messages)

SyncCursorPage[Message](data=[Message(id='msg_o8yXzmcGVloLqV9G1RoMSYhP', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How does memgpt allow LLMs to have unlimited context length?'), type='text')], created_at=1720044430, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_qzRh9tww3y5X0WVK2SKtfp3D'), Message(id='msg_4F39lKrtExI1J905rXw8WQkX', assistant_id=None, attachments=[Attachment(file_id='file-RvM6HYNffOL32MiXJn1T9YQM', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How does memgpt allow LLMs to have unlimited context length?'), type='text')], created_at=1720044430, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_qzRh9tww3y5X0WVK2SK

In [59]:
for message in reversed(messages.data):
    print(message.role + ": " + message.content[0].text.value)

user: Solve this problem: 3x + 11 = 14


In [60]:
file = client.files.create(
    file=open("memgpt-paper.pdf", "rb"),
    purpose= "assistants"
)

In [61]:
print(file)

FileObject(id='file-2cIb3YGXlb8QqZVM69PipxLH', bytes=663708, created_at=1720044425, filename='memgpt-paper.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [2]:
assistant = client.beta.assistants.create(
  name="Machine Learning Researcher",
  instructions="'You are a machine learning researcher, answer questions on the research paper.'",
  model="gpt-3.5-turbo",
  tools=[{"type": "file_search"}],
)

In [3]:
vector_store = client.beta.vector_stores.create(name='memgpt-paper')
file_paths = ['memgpt-paper.pdf']
file_streams = [open(file_path, 'rb') for file_path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_streams,
)
print(file_batch.status)
print(file_batch.file_counts)
print(file_batch)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)
VectorStoreFileBatch(id='vsfb_6f324825a5f44343b99a590d4aa9ce6a', created_at=1720047675, file_counts=FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1), object='vector_store.file_batch', status='completed', vector_store_id='vs_DtRHhJjJ0s0g1Ht0A9tYNzP8')


In [4]:
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)



In [9]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("./memgpt-paper.pdf", "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "How does memgpt allow LLMs to have unlimited context length?",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
print()
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)


ToolResourcesFileSearch(vector_store_ids=['vs_72u2sD9OtclutRQT2B9XTLG3'])


In [10]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = 'user',
    content = 'How does memgpt allow LLMs to have unlimited context length?'
)

In [12]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

IndexError: list index out of range